In [50]:
import torch
from nuclr.data import prepare_nuclear_data
from nuclr.config import config, datadir
from nuclr.loss import loss_by_task, metric_by_task
from nuclr.model import NuCLRWrapper
from nuclr.tensor_dict import TensorDict, Fields
import tqdm

In [51]:
data = prepare_nuclear_data(config)

In [52]:
tensor_dict_train = data.tensor_dict.iloc[data.train_mask]

input_fields = {"n", "z"}
output_fields = set(data.all_fields.keys()) - input_fields
output_dim = sum(data.all_fields[field] for field in output_fields)

fields = Fields({"numerical": list(output_fields), "categorical": []})

In [53]:
d_model = 32
torch.manual_seed(0)
model = torch.nn.Sequential(
    torch.nn.Embedding(sum(data.vocab_size), d_model),
    torch.nn.Flatten(),
    torch.nn.Linear(d_model * 2, d_model),
    torch.nn.ReLU(),
    torch.nn.Linear(d_model, d_model),
    torch.nn.ReLU(),
    torch.nn.Linear(d_model,output_dim),
).to(config.DEV)

model = NuCLRWrapper(model, fields)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)

In [54]:
model(tensor_dict_train[["n", "z"]])

{'binding': tensor([[0.0491],
         [0.1560],
         [0.0769],
         ...,
         [0.1017],
         [0.0494],
         [0.0865]], device='cuda:0', grad_fn=<IndexBackward0>),
 'radius': tensor([[-0.1396],
         [-0.1617],
         [-0.1614],
         ...,
         [-0.1387],
         [-0.1705],
         [ 0.0028]], device='cuda:0', grad_fn=<IndexBackward0>)}

In [55]:
prediction = model(tensor_dict_train[["n", "z"]])
print(metric_by_task(prediction, tensor_dict_train, data.regression_transformers))

{'binding': tensor(249493.2275, dtype=torch.float64), 'radius': tensor(2.5290, dtype=torch.float64)}


In [56]:
for i in (pbar:=tqdm.trange(20000)):
    optimizer.zero_grad()
    losses = (model(X) - y) ** 2
    loss = losses.mean()
    loss.backward()
    optimizer.step()
    pbar.set_description("Loss: {:.4f}".format(loss.item()))
    

  0%|          | 0/20000 [00:00<?, ?it/s]


NameError: name 'X' is not defined